# Projet de Alternative Risk Premia  

* Uftade KARA
* Yann CERASI


#### Importer les modules

In [8]:
import math
import numpy as np, pandas as pd
import statsmodels.regression.linear_model as sm 

####  Variables Global

In [9]:
formPerL = 2 # période de formation du portefeuille = 12 mois ou 2 semestres 
estPerL = 1  # période de placement sur le portefeuille = 6 mois, 1 semestre


### 4.1 Nettoyer et importer les données

##### Veuillez renseigner l'adresse relative du fichier des données dans votre ordinateur
#### Importer le dataset

In [10]:
#Le filePath  
path = "C:\Dauphine Exchange Semester\Risk Premia"
#Import du fichier
CRSP = pd.read_csv(path + "\CRSP data.csv", sep=',')

**Si on utilise le path absolu, on n'a pas besoin d'inscrire le path relatif**

In [11]:
#CRSP = pd.read_csv("CRSP data.csv", sep=',')

In [12]:
#Le Dataframe
CRSP.head()

,PERMNO,date,PRIMEXCH,RET,semester
0,10001,19851231,NaN,NaN,42
1,10001,19860131,Q,C,43
2,10001,19860228,Q,0.020408,43
3,10001,19860331,Q,0.025200,43
4,10001,19860430,Q,0.009901,43


#### explorer le dataframe 

In [13]:
#Chercher des erreurs dans les variables
CRSP[['PERMNO','date','semester']].describe()

,PERMNO,date,semester
count,1.216369e+06,1.216369e+06,1.216369e+06
mean,4.347618e+04,1.979450e+07,3.026773e+01
std,2.069398e+04,6.566477e+04,1.314077e+01
min,1.000100e+04,1.965013e+07,1.000000e+00
25%,2.580600e+04,1.974123e+07,2.000000e+01
50%,4.279600e+04,1.980063e+07,3.100000e+01
75%,5.933700e+04,1.985073e+07,4.200000e+01
max,9.331600e+04,1.989123e+07,5.000000e+01


In [14]:
CRSP[['PRIMEXCH','RET']].describe()

,PRIMEXCH,RET
count,1207297,1187466
unique,4,124065
top,Q,0.000000
freq,573225,73992


In [15]:
CRSP.PRIMEXCH.unique() 


array([nan, 'Q', 'N', 'X', 'A'], dtype=object)

In [16]:
CRSP.semester.unique() 


array([42, 43, 44, 45, 46, 47, 48, 49, 50,  1,  2,  3,  4,  5,  6,  7,  8,
        9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
       26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41],
      dtype=int64)

In [17]:
CRSP.RET.unique() 

array([nan, 'C', '0.020408', ..., '-0.340067', '-0.490385', '1.019231'],
      dtype=object)

#### Utiliser la fonction isin afin de générer un booléen qui donne le résultat True si NYSE ou AMEX, puis construire un nouveaux dataframe qui consiste de ces deux actifs 

In [18]:
stocks = CRSP["PRIMEXCH"].isin(["A","N"]) 
CRSP = CRSP[stocks]
CRSP.head()

,PERMNO,date,PRIMEXCH,RET,semester
147,10006,19650129,N,0.053459,1
148,10006,19650226,N,-0.025970,1
149,10006,19650331,N,0.040248,1
150,10006,19650430,N,0.084821,1
151,10006,19650528,N,-0.043621,1


#### Contrôler si la fonction a donné les résultats souhaités

In [19]:
CRSP.PRIMEXCH.unique() 

array(['N', 'A'], dtype=object)

In [20]:
CRSP.sort_values(by='RET')

,PERMNO,date,PRIMEXCH,RET,semester
760935,50833,19770131,A,-0.000000,25
856426,56434,19730629,A,-0.000000,17
687478,47001,19800630,N,-0.000000,31
660042,45612,19711130,A,-0.000000,14
638050,44345,19780131,N,-0.000000,27
...,...,...,...,...,...
1214157,92567,19891130,N,NaN,50
1214301,92639,19880729,N,NaN,48
1215281,92946,19890929,A,NaN,50
1216118,93172,19860731,A,NaN,44


#### Observer le dtype de RET

In [21]:
CRSP['RET'].value_counts()

0.000000     21537
C             1946
-0.111111     1531
-0.100000     1441
-0.083333     1399
             ...  
-0.022268        1
0.006241         1
-0.027914        1
0.001829         1
0.010378         1
Name: RET, Length: 101899, dtype: int64

#### Constituer un dictionnaire à partir des valeurs non souhaitées et changer le dtype de ces variables

In [22]:
NADict = {-99:np.nan, -88:np.nan, -77:np.nan, -66:np.nan, 'B':np.nan, 'C':np.nan}
CRSP = CRSP.replace({'RET': NADict})

#### Drop tous nan "valeur non defini" dans le dataframe grâce à la méthode dropna 

In [23]:
CRSP.dropna(inplace=True)
CRSP['RET'] = CRSP['RET'].astype(float)
CRSP['PRIMEXCH'] = CRSP['PRIMEXCH'].astype(object)
CRSP.head()

,PERMNO,date,PRIMEXCH,RET,semester
147,10006,19650129,N,0.053459,1
148,10006,19650226,N,-0.025970,1
149,10006,19650331,N,0.040248,1
150,10006,19650430,N,0.084821,1
151,10006,19650528,N,-0.043621,1


In [24]:
CRSP['RET'].value_counts()

 0.000000    21602
-0.111111     1531
-0.100000     1441
-0.083333     1399
-0.125000     1387
             ...  
-0.028179        1
 0.059197        1
-0.172336        1
-0.001116        1
-0.239567        1
Name: RET, Length: 101897, dtype: int64

#### Observer les valeurs minimum et maximum dans un tuple

In [25]:
(CRSP.RET.max(), CRSP.RET.min())

(4.846154, -0.863636)

#### Trier le dataframe par rapport à la colonne RET

In [26]:
CRSP.sort_values(by='RET')

,PERMNO,date,PRIMEXCH,RET,semester
987988,63511,19870630,N,-0.863636,45
1044473,67109,19861231,N,-0.808824,44
802185,53268,19740731,A,-0.808696,20
485822,36214,19740531,A,-0.789474,19
987361,63482,19860530,N,-0.785714,43
...,...,...,...,...,...
139829,17072,19740131,N,3.000000,19
766769,51190,19741031,A,3.166667,20
1005871,64493,19860731,A,3.600000,44
834808,55167,19750131,A,3.600000,21


### 4.2  Constitution des portefeuilles Loosers et Winners

In [27]:
#Retenir les rendements des 12 premiers mois (2 semestres)
formPerioddata = CRSP[CRSP['semester'].between(1, 2, inclusive=True)]
#Grouper les données par PERMNO
by_PERMNO = formPerioddata.groupby('PERMNO')
print(by_PERMNO.head())

         PERMNO      date PRIMEXCH       RET  semester
147       10006  19650129        N  0.053459         1
148       10006  19650226        N -0.025970         1
149       10006  19650331        N  0.040248         1
150       10006  19650430        N  0.084821         1
151       10006  19650528        N -0.043621         1
...         ...       ...      ...       ...       ...
1195675   86239  19650129        N  0.166667         1
1195676   86239  19650226        N  0.010204         1
1195677   86239  19650331        N  0.060606         1
1195678   86239  19650430        N  0.228571         1
1195679   86239  19650528        N -0.062016         1

[9213 rows x 5 columns]


In [28]:
#De se groupby on somme la colonne "RET" pour chaque PERMNO dans la période  de formation
RETform = pd.DataFrame(by_PERMNO[['RET']].sum())
#On renomme la colonne RET à "RET_form" pour le merge plus tard
RETform.columns = ['RET_form']
#On tri les valeurs dans un ordre décroissant
RETform.sort_values(by = 'RET_form', ascending=False, inplace = True)
RETform

,RET_form
PERMNO,
31579,3.069201
38156,2.781328
34569,2.739458
41072,1.966338
28169,1.952610
...,...
39888,-0.369274
35721,-0.432555
14920,-0.467856


**Créer 10 portefeuilles (numéroté de 0 à 9) PF = 9 représente les 10% de rentabilités cumulées les plus fortes.  Ce tri nous donne 376 portefeuilles au total soit 188 portefeuilles pour PF9 et 188 portefeuilles pour PF0.**

In [29]:
formPortfolio = pd.DataFrame(pd.qcut(RETform['RET_form'], q=10, labels=False))
formPortfolio.columns = ['PF_num']
formPortfolio['PF_num'].value_counts()

9    188
4    188
0    188
8    187
7    187
6    187
5    187
3    187
2    187
1    187
Name: PF_num, dtype: int64

In [30]:
formPortfolio

,PF_num
PERMNO,
31579,9
38156,9
34569,9
41072,9
28169,9
...,...
39888,0
35721,0
14920,0


#### Merge les numéros de portefeuille avec RET_form sur PERMNO et utiliser fonction isin pour sélectionner portefeuilles 0 et 9. Créer un nouveau dataframe avec seulement PF = 0 et 9. 

In [31]:
PF_merge = pd.merge(formPortfolio, RETform, how='left', on='PERMNO')
Win_Lose = PF_merge["PF_num"].isin([0,9])
PF_WL = PF_merge[Win_Lose]
PF_WL

,PF_num,RET_form
PERMNO,,
31579,9,3.069201
38156,9,2.781328
34569,9,2.739458
41072,9,1.966338
28169,9,1.952610
...,...,...
39888,0,-0.369274
35721,0,-0.432555
14920,0,-0.467856


## 4.3 Calcul de la rentabilité sur les 6 mois suivants
#### on calcule dèsormais le rendement des PF dans la période holding, après la période de formation de 12 mois. On commence par calculer le rendement cumulé par action sur la période en utilisant la fonction groupby 

In [32]:
estPeriodData = CRSP[CRSP['semester'].between(2, 4, inclusive=False)]
by_PERMNO2 = estPeriodData.groupby('PERMNO')
PFhold = pd.DataFrame(by_PERMNO2[['RET']].sum())
PFhold.columns = ['RET_hold']
PFhold.sort_values(by = 'RET_hold', ascending=False, inplace = True)
PFhold.reset_index(inplace=True) 
PFhold

,PERMNO,RET_hold
0,32352,2.932340
1,29532,1.834482
2,33136,1.610104
3,38770,1.542140
4,36222,1.414053
...,...,...
1843,28177,-0.520717
1844,35641,-0.534234
1845,37217,-0.540022
1846,40811,-0.544258


#### Calculer la rentabilité  moyenne mensuelle par PERMNO

In [33]:
PF_formatest = pd.merge(PF_WL, PFhold, how="left", on='PERMNO')
PF_formatest.sort_values(by = 'RET_hold', inplace = True)
PF_formatest.head()

,PERMNO,PF_num,RET_form,RET_hold
339,40811,0,-0.220553,-0.544258
64,37217,9,1.052460,-0.540022
167,35641,9,0.725091,-0.534234
361,28177,0,-0.332536,-0.520717
358,41021,0,-0.286484,-0.520145


#### Calculer le différentiel des rentabilités sur ces 6 mois entre le portefeuille des Winners et Loosers et ajouter cette valeur à la liste retMom. Utiliser fonction ptp de numpy pour calculer l'écart entre ces deux PF

In [34]:
retMom = []
Data = PF_formatest.groupby('PF_num')['RET_hold'].mean()
Data

PF_num
0   -0.035613
9    0.136269
Name: RET_hold, dtype: float64

In [35]:
r = np.ptp(Data)
retMom.append(r)
retMom

[0.17188154720548263]

#### Le rendement de la stratégie pour la première Itération est de 0,17. Cette valeur est ajoutée à la liste vide retMom. Maintenant il faut itérer sur les autres périodes en décalant de 1 semestre à chaque fois 

## 4.4 Itération et test de significativité


#### 4.2 Période de formation des portefeuilles

In [36]:
def formPeriod(formPerBounds):
    
#retenir rendements des premier 12 mois (2 semestres)
    formPerioddata = CRSP[CRSP['semester'].between(formPerBounds[0], formPerBounds[1], inclusive=True)]
#grouper les données par PERMNO 
    by_PERMNO = formPerioddata.groupby('PERMNO')
#print(by_PERMNO.head())

#De se groupby on somme la colonne RET pour chaque PERMNO dans la période de formation 
    RETform = pd.DataFrame(by_PERMNO[['RET']].sum())
#On renomme la colonne RET a "RET_form" pour le merge plus tard.
    RETform.columns = ['RET_form']
#On tri aussi par le rendement le plus élevé
    RETform.sort_values(by = 'RET_form', ascending=False, inplace = True)

#Créer les 10 portefeuilles (de 0-9)
    formPortfolio = pd.DataFrame(pd.qcut(RETform['RET_form'], q=10, labels=False))
    formPortfolio.columns = ['PF_num']

#merge les numeros des portefeuilles avec RET_form sur PERMNO
    PF_merge = pd.merge(formPortfolio, RETform, how='left', on='PERMNO')

#utiliser fonction isin pour sélectionner portefeuilles 0 et 9. 
    Win_Lose = PF_merge["PF_num"].isin([0,9])
    
#Créer nouveaux dataframe avec seulement PF = 0 et 9.
    PF_WL = PF_merge[Win_Lose]
    formPeriodRet = PF_WL
    
    '''formPeriodRet.()
    
                PF_num RET_form
        PERMNO       
        31579   9     3.069201
        38156   9     2.781328
        34569   9     2.739458
        ...
        39888   0     -0.369274
        
        376 rows '''
    return formPeriodRet

#### 4.3 Période d'estimation de rendement des Portefeuilles

In [37]:
def HoldPeriod(formPerBounds):

#appliquer la méthode groupby pour obtenir les rendements cumulés par indice dans la période
    holdPeriodData = CRSP[CRSP['semester'].between(formPerBounds[1] + 1, formPerBounds[1] + 1, inclusive=True)]
    by_PERMNO2 = holdPeriodData.groupby('PERMNO')
    PFhold = pd.DataFrame(by_PERMNO2[['RET']].sum())
    
#renommer colonne RET a RET_hold, puis reset_index avant le merge dans 4.4    
    PFhold.columns = ['RET_hold']
    PFhold.sort_values(by = 'RET_hold', ascending=True, inplace = True)
    PFhold.reset_index(inplace=True) 
    holdPeriodRet = PFhold

    '''holdPeriodRet()
             RET_hold
    PERMNO
    32352   2.932340
    29532   1.834482
    33136   1.610104
    ...	...
    37217   -0.540022
    40811   -0.544258
    27297   -0.564991

    :1848 rows '''
    return holdPeriodRet

#### Décaler de 6 mois (1 semestre), faire des tests sur la période entière de 50 semestres 


In [38]:
nTests = CRSP['semester'].max()  - (formPerL + estPerL) + 1
nTests

48

### 4.4 Itération

In [39]:
#Iteration à partir de la deuxième itération, la première a été calculé préalablement
for iter in range(2, nTests + 1):
    print("\n ****** Iteration: " + str(iter) + "/" + str(nTests) + " ******\n")

# définir la période de formation 
    formPerBounds = (iter, iter + formPerL - 1)
    print("formation period: " + str(formPerBounds)) # tuple iterates period (1,2) (3)
    formPeriodRet = formPeriod(formPerBounds)

# définir la période "holding" 
    print("estimation period: " + str(formPerBounds[1] + 1))## tuple period (2,4) not inclusive so =(3)
    holdPeriodRet = HoldPeriod(formPerBounds)
    
#Merge les indices avec les numéros de portefeuilles construit dans la période de formation
    PF_formatest = pd.merge(formPeriodRet, holdPeriodRet, how="left", on='PERMNO')
    
#Calculer le taux de rentabilité moyen mensuel de chaque portefeuille dans la période "Hold" 
    Data = PF_formatest.groupby('PF_num')['RET_hold'].mean()
    
#Calculer le différentiel des rentabilités      
    r = np.ptp(Data)
    retMom.append(r)
    
#Retmom inclus la première observation imprimée avant l'itération 
    print(retMom)


 ****** Iteration: 2/48 ******

formation period: (2, 3)
estimation period: 4
[0.17188154720548263, 0.02151049456521742]

 ****** Iteration: 3/48 ******

formation period: (3, 4)
estimation period: 5
[0.17188154720548263, 0.02151049456521742, 0.10115930781954846]

 ****** Iteration: 4/48 ******

formation period: (4, 5)
estimation period: 6
[0.17188154720548263, 0.02151049456521742, 0.10115930781954846, 0.12013144827059874]

 ****** Iteration: 5/48 ******

formation period: (5, 6)
estimation period: 7
[0.17188154720548263, 0.02151049456521742, 0.10115930781954846, 0.12013144827059874, 0.1527784474768855]

 ****** Iteration: 6/48 ******

formation period: (6, 7)
estimation period: 8
[0.17188154720548263, 0.02151049456521742, 0.10115930781954846, 0.12013144827059874, 0.1527784474768855, 0.15659761821283588]

 ****** Iteration: 7/48 ******

formation period: (7, 8)
estimation period: 9
[0.17188154720548263, 0.02151049456521742, 0.10115930781954846, 0.12013144827059874, 0.1527784474768855

estimation period: 30
[0.17188154720548263, 0.02151049456521742, 0.10115930781954846, 0.12013144827059874, 0.1527784474768855, 0.15659761821283588, 0.10867358910325764, 0.04693927000000005, 0.0968314239813417, 0.11162654831552521, 0.07019972249546302, 0.06551221390159317, 0.029861662107876386, 0.037485971360895765, 0.12526228244930454, 0.20094649702850967, 0.2532310145932605, 0.3481313992346936, 0.4122838401149533, 0.0008567845021467679, 0.021558441161121877, 0.010899214853938466, 0.16799990630027395, 0.04226946543778801, 0.061264791577749445, 0.07694404040404042, 0.050052107813569335, 0.12271268857888395]

 ****** Iteration: 29/48 ******

formation period: (29, 30)
estimation period: 31
[0.17188154720548263, 0.02151049456521742, 0.10115930781954846, 0.12013144827059874, 0.1527784474768855, 0.15659761821283588, 0.10867358910325764, 0.04693927000000005, 0.0968314239813417, 0.11162654831552521, 0.07019972249546302, 0.06551221390159317, 0.029861662107876386, 0.037485971360895765, 0.125262

[0.17188154720548263, 0.02151049456521742, 0.10115930781954846, 0.12013144827059874, 0.1527784474768855, 0.15659761821283588, 0.10867358910325764, 0.04693927000000005, 0.0968314239813417, 0.11162654831552521, 0.07019972249546302, 0.06551221390159317, 0.029861662107876386, 0.037485971360895765, 0.12526228244930454, 0.20094649702850967, 0.2532310145932605, 0.3481313992346936, 0.4122838401149533, 0.0008567845021467679, 0.021558441161121877, 0.010899214853938466, 0.16799990630027395, 0.04226946543778801, 0.061264791577749445, 0.07694404040404042, 0.050052107813569335, 0.12271268857888395, 0.11983892316784875, 0.18631863086084696, 0.18567255957334855, 0.01849324496880539, 0.11303849425758823, 0.05309175436305008, 0.07682683310945276, 0.009821615948646334, 0.002400930447433558, 0.2002759977998696, 0.048983313738459294, 0.19325066779493327, 0.22549713226010099]

 ****** Iteration: 42/48 ******

formation period: (42, 43)
estimation period: 44
[0.17188154720548263, 0.02151049456521742, 0.10115

#### Transformer retMom en dataframe, puis renommer la colonne à RETStrat, utilisation de describe pour les statistiques descriptives 

In [40]:
retMom = pd.DataFrame(retMom)
retMom.columns=["RETStrat"]
retMom.describe()

,RETStrat
count,48.000000
mean,0.110330
std,0.091216
min,0.000857
25%,0.041074
50%,0.098995
75%,0.168970
max,0.412284


### Le rendement moyen de la stratégie est de 0,11 avec un écartype de 0,09. Il faut un t-test pour déterminer la significativité de la stratégie momentum 

#### Le t-test et p-value pour décider si ce rendement est significatif 

In [41]:
t_stat=retMom['RETStrat'].mean() * math.sqrt(retMom['RETStrat'].count()) / retMom['RETStrat'].std()
print("T-test: " + str(t_stat))

T-test: 8.379954186633785


In [42]:
#Df
df = retMom['RETStrat'].count() - 1
#Alpha
alpha=0.05
#Import student-t distribution
from scipy.stats import t

In [43]:
#Calculer p-values
p_value = (1 - t.cdf(abs(t_stat), df)) * 2
if p_value < alpha:
    print('p-value:  ' + str(p_value) + '\nReject Null Hypothesis of no significant effect')
else:
    print('p-value:  ' + str(p_value) + '\nKeep Null Hypothesis of no significant effect')

p-value:  6.942890706795879e-11
Reject Null Hypothesis of no significant effect


#### On conclue donc que la stratégie momentum Buy-sell proposé par Jegadeesh et Titman donne un rendement significatif. 